<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB02_LLM_Cycles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Cycles de "prompt-réponse" avec un LLM

**Objectifs :**  
- Comprendre et expérimenter la manière dont un LLM traite les invites,
- Comment il conserve le contexte
- et comment il produit des réponses lors de plusieurs échanges.

**Durée estimée :** 75–90 minutes

**Livrable :** Notebook illustrant les cycles invite-réponse à un seul tour, à plusieurs tours et avec réinitialisation du contexte.

---

## Step 1: Pré-requis (5 min)

Installation des dépendances nécessaires et configuration de l'API OpenAI.

In [1]:
# Installation des dépendances
!pip install openai python-dotenv -q
print("✓ Dépendances installées avec succès")

✓ Dépendances installées avec succès


In [9]:
# Configuration de l'API OpenAI
import os
from google.colab import userdata

# Get API key from Colab Secrets (add it in the Secrets manager: 🔑 icon on left panel)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Or if running locally with .env file:
# from dotenv import load_dotenv
# load_dotenv()

print("✓ API Key configured successfully")



✓ API Key configured successfully


## Step 2: Définition d'une fonction auxiliaire "chat_cycle" (10 min)

Créer une fonction réutilisable pour les cycles conversation.

In [12]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()


def chat_cycle(messages, model="gpt-4o-mini", temperature=0.7):
    """
    Exécute un cycle de chat avec l'API OpenAI.

    Args:
        messages (list): Liste des messages {"role": ..., "content": ...}
        model (str): Modèle à utiliser (défaut: gpt-4o-mini)
        temperature (float): Contrôle la créativité (0.0-1.0)

    Returns:
        str: Contenu de la réponse du modèle
    """
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature
        )
        return resp.choices[0].message.content
    except Exception as e:
        return f"Erreur: {str(e)}"

print("✓ Fonction chat_cycle() définie")

✓ Fonction chat_cycle() définie


## Step 3: Single-Turn Prompting (10 min)

Tester les prompts simples et observer la variabilité.

In [13]:
print("=" * 60)
print("SINGLE-TURN PROMPTING: Test 1 - Prompt sans contrainte")
print("=" * 60)

messages = [{"role": "user", "content": "Explain Agentic AI in one sentence."}]
response1 = chat_cycle(messages)
print(f"\nRun 1:\n{response1}")

# Relancer pour observer la variabilité
response2 = chat_cycle(messages)
print(f"\nRun 2 (même prompt):\n{response2}")

print(f"\n📌 Observation: Les réponses sont-elles identiques? {response1 == response2}")

SINGLE-TURN PROMPTING: Test 1 - Prompt sans contrainte

Run 1:
Agentic AI refers to artificial intelligence systems that possess the ability to make independent decisions and take actions in pursuit of specific goals, often exhibiting a level of autonomy and adaptability in their operations.

Run 2 (même prompt):
Agentic AI refers to artificial intelligence systems that possess the capacity to act autonomously and make decisions based on their own assessments of situations, rather than merely following pre-defined instructions.

📌 Observation: Les réponses sont-elles identiques? False


In [14]:
print("\n" + "=" * 60)
print("SINGLE-TURN PROMPTING: Test 2 - Prompt avec contrainte")
print("=" * 60)

messages_constrained = [
    {"role": "user", "content": "Explain Agentic AI in exactly 10 words. Count your words."}
]
response_constrained = chat_cycle(messages_constrained)
print(f"\nRéponse:\n{response_constrained}")

word_count = len(response_constrained.split())
print(f"\n📌 Nombre de mots: {word_count}")


SINGLE-TURN PROMPTING: Test 2 - Prompt avec contrainte

Réponse:
Agentic AI possesses autonomy, making independent decisions in specific contexts.

📌 Nombre de mots: 10


## Step 4: Multi-Turn Conversations (15 min)

Expérimenter avec plusieurs tours et observer la persistance du contexte.

In [15]:
print("=" * 60)
print("MULTI-TURN CONVERSATION: Building Context")
print("=" * 60)

# Initialiser la conversation avec un système prompt
conversation = [
    {"role": "system", "content": "You are a friendly teaching assistant specializing in AI and distributed systems."},
    {"role": "user", "content": "What is an AI agent?"}
]

# Tour 1
print("\n--- Turn 1 ---")
print("User: What is an AI agent?")
reply1 = chat_cycle(conversation)
print(f"\nAssistant: {reply1}")

# Ajouter la réponse à la conversation
conversation.append({"role": "assistant", "content": reply1})
conversation.append({"role": "user", "content": "Can you give me an example in healthcare?"})

# Tour 2
print("\n--- Turn 2 ---")
print("User: Can you give me an example in healthcare?")
reply2 = chat_cycle(conversation)
print(f"\nAssistant: {reply2}")

# Ajouter pour le tour 3
conversation.append({"role": "assistant", "content": reply2})
conversation.append({"role": "user", "content": "How does this relate to what you mentioned earlier?"})

# Tour 3
print("\n--- Turn 3 ---")
print("User: How does this relate to what you mentioned earlier?")
reply3 = chat_cycle(conversation)
print(f"\nAssistant: {reply3}")

print("\n📌 Observation: L'assistant maintient-il le contexte des tours précédents? Oui!")

MULTI-TURN CONVERSATION: Building Context

--- Turn 1 ---
User: What is an AI agent?

Assistant: An AI agent is a system that can perceive its environment, make decisions, and take actions to achieve specific goals based on the information it gathers. These agents can operate autonomously or semi-autonomously and are designed to solve problems or perform tasks that typically require human intelligence.

Key characteristics of AI agents include:

1. **Perception**: AI agents can observe their environment using sensors or data inputs (e.g., cameras, microphones, or data streams).

2. **Reasoning and Decision-Making**: They process the information they perceive to reason about their environment and make decisions. This often involves using algorithms, heuristics, or machine learning techniques.

3. **Action**: Once a decision is made, the agent can take actions in the environment, which can include physical movements (in the case of robots) or digital actions (like sending messages or mak

## Step 5: Context Window Experiment (15 min)

Tester les limites de la fenêtre de contexte.

In [16]:
print("=" * 60)
print("CONTEXT WINDOW EXPERIMENT: Testing Limits")
print("=" * 60)

# Créer un long article à injecter
long_article = """
# The Evolution of Artificial Intelligence

Artificial Intelligence (AI) has evolved dramatically over the past six decades. From the early days of
symbolic reasoning and expert systems in the 1970s, through the neural network revolution of the 2010s,
to the large language models of today, AI has transformed from theoretical computer science to practical
tools that power our digital lives.

The journey began with Alan Turing's question: "Can machines think?" This simple question sparked decades
of research. Early AI systems like ELIZA and SHRDLU demonstrated that machines could simulate conversation
and understand limited domains. Expert systems of the 1980s brought AI to industry, encoding human expertise
in rule-based systems.

The winter periods—times when funding dried up and expectations outpaced capabilities—taught the field
valuable lessons about the dangers of overpromising. Yet each winter was followed by spring: new paradigms
emerged, new data became available, and computing power increased exponentially.

The deep learning revolution of the 2010s changed everything. Convolutional neural networks conquered image
recognition. Recurrent networks learned language. Transformers, introduced in 2017, enabled the scale of
models we see today. GPT models, BERT, and their successors demonstrated that scaling up language models
on internet-scale data could produce surprisingly capable systems.

Today, we stand at an inflection point. Large language models can write code, explain complex concepts,
and engage in nuanced reasoning. Multimodal models understand both images and text. And yet, current systems
still struggle with tasks that humans find trivial: reasoning about causality, understanding physical
intuition, and maintaining consistency over long documents.

The next frontiers are agentic AI—systems that can plan, act, and iterate—and multiagent systems where
multiple AI entities collaborate and compete. These developments promise to make AI more useful, but also
raise important questions about alignment, safety, and societal impact.
"""

context_conversation = [
    {"role": "system", "content": "You are a knowledgeable AI assistant."},
    {"role": "user", "content": f"Please read this article carefully:\n{long_article}\n\nWhen you're done, just say 'Article read and understood.'"}
]

print("\n[Injecting long article...]")
ack = chat_cycle(context_conversation)
print(f"Assistant: {ack}")

context_conversation.append({"role": "assistant", "content": ack})

CONTEXT WINDOW EXPERIMENT: Testing Limits

[Injecting long article...]
Assistant: Article read and understood.


In [17]:
# Maintenant, poser plusieurs questions pour voir où le contexte s'estompe
print("\n--- Question 1: Début de l'article ---")
q1 = "Who first asked 'Can machines think'?"
context_conversation.append({"role": "user", "content": q1})
reply_q1 = chat_cycle(context_conversation)
print(f"Q: {q1}")
print(f"A: {reply_q1}")
context_conversation.append({"role": "assistant", "content": reply_q1})

print("\n--- Question 2: Milieu de l'article ---")
q2 = "What was the deep learning revolution?"
context_conversation.append({"role": "user", "content": q2})
reply_q2 = chat_cycle(context_conversation)
print(f"Q: {q2}")
print(f"A: {reply_q2}")
context_conversation.append({"role": "assistant", "content": reply_q2})

print("\n--- Question 3: Fin de l'article ---")
q3 = "What are the next frontiers mentioned in the article?"
context_conversation.append({"role": "user", "content": q3})
reply_q3 = chat_cycle(context_conversation)
print(f"Q: {q3}")
print(f"A: {reply_q3}")

print("\n📌 Observation: Le modèle se souvient-il de détails du début, du milieu, et de la fin?")


--- Question 1: Début de l'article ---
Q: Who first asked 'Can machines think'?
A: The question "Can machines think?" was first posed by Alan Turing.

--- Question 2: Milieu de l'article ---
Q: What was the deep learning revolution?
A: The deep learning revolution refers to a significant advancement in artificial intelligence that began in the early 2010s, characterized by the development and application of deep neural networks. This revolution transformed various fields, particularly in areas like image and language processing.

Key aspects include:

1. **Convolutional Neural Networks (CNNs)**: These networks excelled at image recognition tasks, significantly improving the accuracy of computer vision applications.

2. **Recurrent Neural Networks (RNNs)**: These networks were particularly effective in processing sequential data, making them suitable for tasks involving language, such as translation and speech recognition.

3. **Transformers**: Introduced in 2017, transformers revoluti

## Step 6: Controlled Output Cycles (15 min)

Tester les formats de sortie structurés.

In [18]:
print("=" * 60)
print("CONTROLLED OUTPUT CYCLES: Testing Formats")
print("=" * 60)

# Test 1: JSON
print("\n--- Test 1: JSON Format ---")
messages_json = [
    {"role": "system", "content": "You are a JSON-only assistant. Always respond with valid JSON."},
    {"role": "user", "content": "List 3 tools used in Agentic AI in JSON format with 'name' and 'description' fields."}
]
response_json = chat_cycle(messages_json)
print(response_json)

CONTROLLED OUTPUT CYCLES: Testing Formats

--- Test 1: JSON Format ---
```json
[
    {
        "name": "Reinforcement Learning",
        "description": "A type of machine learning where agents learn to make decisions by receiving rewards or penalties based on their actions, enabling them to optimize performance over time."
    },
    {
        "name": "Natural Language Processing (NLP)",
        "description": "A field of AI that focuses on the interaction between computers and humans through natural language, enabling agents to understand, interpret, and respond to human language."
    },
    {
        "name": "Computer Vision",
        "description": "An area of AI that enables agents to interpret and understand visual information from the world, allowing them to recognize objects, track movements, and analyze images."
    }
]
```


In [19]:
# Test 2: Bullet points
print("\n--- Test 2: Bullet List Format ---")
messages_bullet = [
    {"role": "system", "content": "You are a bullet-point expert. Always respond using bullet points."},
    {"role": "user", "content": "List 3 tools used in Agentic AI as bullet points."}
]
response_bullet = chat_cycle(messages_bullet)
print(response_bullet)


--- Test 2: Bullet List Format ---
- **Reinforcement Learning Frameworks**: Tools like OpenAI Gym and TensorFlow Agents that enable agents to learn optimal behaviors through trial and error.
- **Natural Language Processing Libraries**: Libraries such as Hugging Face Transformers and SpaCy for understanding and generating human language.
- **Simulation Environments**: Platforms like Unity ML-Agents and Gazebo that provide realistic scenarios for training and testing AI agents.


In [20]:
# Test 3: Table format
print("\n--- Test 3: Table Format (Markdown) ---")
messages_table = [
    {"role": "system", "content": "You are a table expert. Always respond using Markdown tables."},
    {"role": "user", "content": "Create a table with 3 agentic AI tools, their main features, and use cases."}
]
response_table = chat_cycle(messages_table)
print(response_table)

print("\n📌 Observation: Quel format est le plus facilement parsable par un programme?")


--- Test 3: Table Format (Markdown) ---
Here's a table detailing three agentic AI tools, their main features, and their use cases:

| AI Tool          | Main Features                                     | Use Cases                                         |
|------------------|---------------------------------------------------|--------------------------------------------------|
| OpenAI GPT-3     | - Natural language understanding                   | - Content generation (blogs, articles, etc.)    |
|                  | - Conversational capabilities                       | - Customer support (chatbots)                    |
|                  | - Text summarization                               | - Virtual assistants                               |
|                  | - Language translation                             |                                                  |
|------------------|---------------------------------------------------|--------------------------------------------

## Step 7: Mini Project - Q&A Agent Simulation (15-20 min)

Construire une boucle interactive qui maintient le contexte.

In [21]:
print("=" * 60)
print("MINI PROJECT: Interactive Q&A Agent")
print("=" * 60)
print("\nCet agent éducatif maintient la contexte à travers plusieurs questions.")
print("Tapez 'exit' ou 'quit' pour terminer.")
print("Posez des questions liées pour voir le maintien du contexte!\n")

# Initialiser l'agent avec un contexte
agent_conversation = [
    {
        "role": "system",
        "content": "You are an educational AI tutor specializing in agentic AI, RAG systems, and distributed computing. Be clear, concise, and build on previous questions to show continuity."
    }
]

def interactive_qa_agent():
    """
    Boucle interactive pour le Q&A agent.
    """
    turn_count = 0

    while True:
        turn_count += 1

        # Simuler les entrées utilisateur avec des questions prédéfinies
        sample_questions = [
            "What is RAG (Retrieval-Augmented Generation)?",
            "How does memory help an AI agent?",
            "Give me an analogy relating RAG to something in daily life.",
            "How would you combine these concepts in a real system?"
        ]

        if turn_count <= len(sample_questions):
            user_input = sample_questions[turn_count - 1]
            print(f"\n[Turn {turn_count}]")
            print(f"👤 User: {user_input}")
        else:
            # Après les questions prédéfinies, permettre l'entrée utilisateur
            user_input = input("\n👤 User: ").strip()
            if not user_input:
                continue
            if user_input.lower() in ["exit", "quit"]:
                print("\n✓ Agent simulation ended. Goodbye!")
                break

        # Ajouter la question à la conversation
        agent_conversation.append({"role": "user", "content": user_input})

        # Obtenir la réponse
        reply = chat_cycle(agent_conversation)
        print(f"🤖 AI: {reply}")

        # Ajouter la réponse à la conversation pour le contexte suivant
        agent_conversation.append({"role": "assistant", "content": reply})

        # Arrêter après les questions prédéfinies pour éviter une boucle infinie en Colab
        if turn_count >= len(sample_questions):
            print("\n" + "="*60)
            print("Questions prédéfinies terminées.")
            print("Vous pouvez continuer à poser des questions ou taper 'exit'.")
            print("="*60)

# Lancer l'agent
interactive_qa_agent()

MINI PROJECT: Interactive Q&A Agent

Cet agent éducatif maintient la contexte à travers plusieurs questions.
Tapez 'exit' ou 'quit' pour terminer.
Posez des questions liées pour voir le maintien du contexte!


[Turn 1]
👤 User: What is RAG (Retrieval-Augmented Generation)?
🤖 AI: Retrieval-Augmented Generation (RAG) is a hybrid approach that combines the strengths of information retrieval and natural language generation. In RAG systems, a model retrieves relevant documents or information from a large corpus based on a query and then uses that information to generate a more informed and contextually relevant response.

The RAG architecture typically involves two main components:

1. **Retriever**: This component searches a knowledge base or a document corpus to find relevant pieces of information related to a user's query. It employs techniques such as TF-IDF, BM25, or more advanced neural retrieval methods.

2. **Generator**: After retrieving the relevant documents, the generator process

## Summary & Key Learnings

Félicitations! Vous avez complété le Lab 2. Voici ce que vous avez appris:

In [23]:
summary = """
📚 KEY LEARNINGS FROM LAB 2
═════════════════════════════════════════════════════════════

1️⃣  SINGLE-TURN PROMPTING
   • Les LLMs produisent des réponses variées même pour un même prompt
   • Les contraintes ("exactly N words") augmentent la prévisibilité
   • La température (temperature parameter) permet de contrôler la "créativité"

2️⃣  MULTI-TURN CONVERSATIONS
   • Les LLMs maintiennent le contexte tout au long d'échanges successifs
   • L'historique du message (message list) est crucial
   • Ajouter les réponses antérieures à la liste améliore la cohérence

3️⃣  CONTEXT WINDOW LIMITS
   • Les LLMs ont une fenêtre de contexte maximale
   • gpt-4o-mini supporte ~128K tokens (très large)
   • Les vieux messages peuvent être perdus dans de très longs contextes

4️⃣  STRUCTURED OUTPUTS
   • Les instructions système contrôlent le format de sortie
   • JSON > Markdown tables pour le parsing programmatique
   • Les contraintes de format améliorent la prévisibilité

5️⃣  AGENTIC PATTERNS
   • Les boucles conversationnelles peuvent simuler des agents
   • La mémoire (conversation history) = fondement des agents
   • Itération + contexte = capacité de raisonnement améliorée

═════════════════════════════════════════════════════════════

🎯 NEXT STEPS:
   • Explorer system prompts plus sophistiqués
   • Implémenter des boucles de réflexion (chain-of-thought)
   • Construire des agents avec des outils/actions
   • Intégrer une mémoire persistante (RAG, embeddings)
"""

print(summary)


📚 KEY LEARNINGS FROM LAB 2
═════════════════════════════════════════════════════════════

1️⃣  SINGLE-TURN PROMPTING
   • Les LLMs produisent des réponses variées même pour un même prompt
   • Les contraintes ("exactly N words") augmentent la prévisibilité
   • La température (temperature parameter) permet de contrôler la "créativité"

2️⃣  MULTI-TURN CONVERSATIONS
   • Les LLMs maintiennent le contexte tout au long d'échanges successifs
   • L'historique du message (message list) est crucial
   • Ajouter les réponses antérieures à la liste améliore la cohérence

3️⃣  CONTEXT WINDOW LIMITS
   • Les LLMs ont une fenêtre de contexte maximale
   • gpt-4o-mini supporte ~128K tokens (très large)
   • Les vieux messages peuvent être perdus dans de très longs contextes

4️⃣  STRUCTURED OUTPUTS
   • Les instructions système contrôlent le format de sortie
   • JSON > Markdown tables pour le parsing programmatique
   • Les contraintes de format améliorent la prévisibilité

5️⃣  AGENTIC PATTERNS

In [24]:
print("\n✅ Lab 2 Complété!")
print("Vous avez acquis une compréhension solide des cycles prompt-response.")
print("Ces concepts sont fondamentaux pour construire des systèmes agentic avancés.")


✅ Lab 2 Complété!
Vous avez acquis une compréhension solide des cycles prompt-response.
Ces concepts sont fondamentaux pour construire des systèmes agentic avancés.
